# Heuristic Functions
There are two heuristics:
- h1 - Heuristic using Manhattan distance
- h2 - Heuristic using number of misplaced tiles

0 represents a blank tile.

In [8]:
import numpy as np
from copy import deepcopy

In [9]:
def mat_to_dict(matrix):
    """
    Converts goal state to dictionary to get indices easier
    
    Parameters:
    matrix(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    
    Returns:
    mat_dict(dict[int, tuple[int, int]]): A mapping of cell contents to a tuple
    contianing its indices
    """
    mat_dict = {}
    for i in range(matrix.shape[0]):
        row = matrix[i];
        for j in range(matrix.shape[1]):
            value = row[j];
            mat_dict[value] = (i, j)
    return mat_dict

In [10]:
def h1(curr_state, goal_dict):
    """
    Heuristic for calculating the distance of goal state using Manhattan distance
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    goal_dict(dict[int, tuple[int, int]]): A mapping of cell contents to a tuple
    contianing its indices
    
    Returns:
    h(int): Heuristic value
    """
    h = 0
    for i in range(curr_state.shape[0]):
        for j in range(curr_state.shape[1]):
            value = curr_state[i][j]
            x = goal_dict[value][0]
            y = goal_dict[value][1]
            h += abs(i-x) + abs(j-y)
    return h

In [11]:
def h2(curr_state, goal_dict):
    """
    Heuristic for calculating the distance of goal state using number of misplaced tiles
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    goal_dict(dict[int, tuple[int, int]]): A mapping of cell contents to a tuple
    contianing its indices
    
    Returns:
    h(int): Heuristic value
    """
    h = 0
    for i in range(curr_state.shape[0]):
        for j in range(curr_state.shape[1]):
            value = curr_state[i][j]
            x = goal_dict[value][0]
            y = goal_dict[value][1]
            if ((x, y) != (i, j)):
                h += 1
    return h

In [12]:
GOAL_STATE = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print("Goal State:\n", GOAL_STATE)

Goal State:
 [[1 2 3]
 [8 0 4]
 [7 6 5]]


In [13]:
goal_dict = mat_to_dict(GOAL_STATE)
print("Goal dict:", goal_dict)

Goal dict: {1: (0, 0), 2: (0, 1), 3: (0, 2), 8: (1, 0), 0: (1, 1), 4: (1, 2), 7: (2, 0), 6: (2, 1), 5: (2, 2)}


In [14]:
# Change this to test
CURR_STATE = np.array([[2, 3, 0], [1, 8, 4], [7, 6, 5]])
print("Current State:\n", CURR_STATE)

Current State:
 [[2 3 0]
 [1 8 4]
 [7 6 5]]


In [15]:
# Heuristic 1
print("Value from h1:", h1(CURR_STATE, goal_dict))

Value from h1: 6


In [16]:
# Heuristic 2
print("Value from h2:", h2(CURR_STATE, goal_dict))

Value from h2: 5


In [17]:
def find_pos(matrix,value):
    if value < 0 or value > 8:
        raise Exception ("Give the value is out of range")
    else:
        for row in range(len(matrix)):
            for col in range(len(matrix)):
                if (matrix[row][col] == value):
                    return row, col

In [18]:

def possible_moves(matrix):
    row,col = find_row_col(matrix,0)
    possible_moves = list()
    if row > 0:
                possible_moves.append((row-1,col))
    if row < 2:
                possible_moves.append((row+1,col))
    if col > 0:
                possible_moves.append((row,col-1))
    if col < 2:
                possible_moves.append((row,col+1))
    return possible_moves

In [19]:
def swap_tiles(matrix,next_pos):
    row,col=find_pos(matrix,0)
    blank_tile=matrix[row][col]
    swap_tile=matrix[next_pos[0]][next_pos[1]]
    matrix[row][col]=swap_tile
    matrix[next_pos[0]][next_pos[1]]=blank_tile
    return matrix


In [20]:
def possibe_moves(current_state):
    row,col=find_pos(matrix,0)
    ans_list=[]
    if(row>0):
        ans_list.append((row-1,col))
    if(row<2):
        ans_list.append((row+1,col))
    if(col>0):
        ans_list.append((row,col-1))
    if(col<2):
        ans_list.append((row,col+1))
    return ans_list

In [6]:
def solve(curr_state):
    next_possible=[]
    heurestic_val=[]
    choosen_matrix=curr_state
    choosen_dict = mat_to_dict(choosen_matrix)
    if (chosen_dict == goal_dict):
        print("REACHED THE GOAL STATE")
        return 
    for i in possible_moves(curr_state):
        matrix2=deepcopy(curr_state)
        temp=swap_tiles(matrix2,i)
        next_possible.append(temp)

        c=h1(temp,GOAL_STATE) # calculate herestic val
        heurestic_val.append(c)
    for k in range(len(next_possible)):
        if(heurestic_val[k]==min(heurestic_val)):
            choosen_matrix=next_possible[k]
    print(choosen_matrix)
    choosen_dict = mat_to_dict(choosen_matrix)
    if (chosen_dict == goal_dict):
        print("REACHED THE GOAL STATE")
    else:
        solve(curr_state)


In [21]:
solve( [[1,2,3],[4,5,6],[0,7,8]])

AttributeError: 'list' object has no attribute 'shape'